CSCI2000U final project group 9:
    Paul Hrunda, Garrett Kawabata, Anthony Liscio

In [161]:
import csv
import numpy as np
import pandas as pd

def get_data_csv():
    collection = []
    with open('imdb_top_1000.csv', 'r') as f:
        for line in csv.DictReader(f):
            collection.append(line)
        return collection
        
# the data    
data = get_data_csv()

data_df = pd.DataFrame(data)

# **1.**
**What is the average IMDB rating per genre? What are the top three?**

In [162]:
# the different genres
total_genres = []
genres = []

def fill_list_total(list_to_fill, key):   
    for i in range(len(data)):
        if data[i][key]:
            list_to_fill.append(data[i][key])

def fill_unique(list_to_fill, total_list):
    for i in total_list:
        if i not in list_to_fill:
            list_to_fill.append(i)

fill_list_total(total_genres, "Genre")
total_genres = [line.split(',')[0] for line in total_genres] # many movies have more than one genre which are seperated
                                                             # by comma, so this just removes the commas and splits the data into their own elements
fill_unique(genres, total_genres)

print("There are "+str(len(genres))+" different game genres:")
print(genres)

There are 14 different game genres:
['Drama', 'Crime', 'Action', 'Biography', 'Western', 'Comedy', 'Adventure', 'Animation', 'Horror', 'Mystery', 'Film-Noir', 'Fantasy', 'Family', 'Thriller']


In [163]:
def get_imdb_by_genre(genre):
    return data_df['IMDB_Rating'][data_df['Genre'].str.contains(genre, regex=False)]

imdb_by_genre = pd.DataFrame(index=range(len(data_df)),columns=range(len(genres)))
imdb_by_genre.columns = genres

for i in genres:
    imdb_by_genre[i] = get_imdb_by_genre(i)
    imdb_by_genre[i] = imdb_by_genre[i].astype(float)

avg_imdb_by_genre = imdb_by_genre.mean(axis = 0, skipna = True)

print('Average IMDB ratings (by genre):')
print(avg_imdb_by_genre.round(3))
print('\nTop 3 IMDB rating (by genre):\n'+str(avg_imdb_by_genre.sort_values(ascending = False).round(3)[:3]))

Average IMDB ratings (by genre):
Drama       7.96
Crime       7.96
Action      7.95
Biography   7.94
Western     8.00
Comedy      7.90
Adventure   7.95
Animation   7.93
Horror      7.89
Mystery     7.97
Film-Noir   7.99
Fantasy     7.93
Family      7.91
Thriller    7.91
dtype: float64

Top 3 IMDB rating (by genre):
Western     8.00
Film-Noir   7.99
Mystery     7.97
dtype: float64


# **2.**
**Does the runtime correlate to the IMDB rating?**

# **3.**
**Do any directors appear more than once? If so who appears most often?**

In [164]:
director_uni = data_df['Director'].nunique()
print("There are ", director_uni, " different directors")

There are  548  different directors


In [165]:
data_df['Movie_Count'] = 1

director_stats = data_df[['Director', 'Movie_Count']].groupby(['Director']).sum()
data_df['IMDB_Rating'] = data_df['IMDB_Rating'].astype(float)
director_stats['IMDB_Rating'] = data_df[['Director','IMDB_Rating']].groupby(['Director']).mean()
director_stats = director_stats.rename(columns={"IMDB_Rating" : "avg IMDB_Rating"})

data_df['No_of_Votes'] = data_df['No_of_Votes'].astype(float)
director_stats['No_of_Votes'] = data_df[['Director','No_of_Votes']].groupby(['Director']).mean()

director_stats = director_stats.rename(columns={"No_of_Votes" : "avg No_of_Votes"})
pd.options.display.float_format = "{:,.2f}".format

In [166]:
#There are directors who appear more than once 
#Top 10 Directors by Number of Movies in the list
print("Top 10 Directors by Number of Movies:")
director_stats = director_stats.sort_values(by=['Movie_Count'], ascending=False)
director_stats.head(10)

Top 10 Directors by Number of Movies:


,Movie_Count,avg IMDB_Rating,avg No_of_Votes
Director,,,
Alfred Hitchcock,14,8.01,"192,612.64"
Steven Spielberg,13,8.03,"601,320.46"
Hayao Miyazaki,11,8.02,"213,519.82"
Martin Scorsese,10,8.17,"651,353.00"
Akira Kurosawa,10,8.22,"94,159.90"
Stanley Kubrick,9,8.23,"435,472.67"
Woody Allen,9,7.79,"135,834.22"
Billy Wilder,9,8.14,"115,322.56"
Quentin Tarantino,8,8.18,"1,015,401.00"


# **4.**
**What is the average meta score rating per genre? What are the top five?**

# **5.**
**What year has the highest meta score?**

# **6.**
**Runtime per certification?**

In [167]:
cert_stats = data_df[['Certificate', 'Runtime']]
cert_stats['Runtime'] = cert_stats['Runtime'].map(lambda x: x.rstrip(' min'))
cert_stats['Runtime'] = cert_stats['Runtime'].astype(int)
cert_stats = cert_stats[['Certificate', 'Runtime']].groupby(['Certificate']).mean()
cert_stats = cert_stats.rename(columns={"Runtime" : "avg Runtime (min)"})
cert_stats

/tmp/ipykernel_44/4145960008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cert_stats['Runtime'] = cert_stats['Runtime'].map(lambda x: x.rstrip(' min'))
/tmp/ipykernel_44/4145960008.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cert_stats['Runtime'] = cert_stats['Runtime'].astype(int)


,avg Runtime (min)
Certificate,
,111.18
16,130.00
A,128.94
Approved,111.09
G,119.67
GP,124.50
PG,114.27
PG-13,116.95
Passed,106.21


# **7.** 
**What is the shortest runtime? What is the longest runtime? Which one has the higher IMDB rating? Which one has the higher meta score?
Follow up questions: What is the runtime of the movie with the highest meta score? How does it compare to the highest/lowest runtime movies?**

In [168]:
data_df['Runtime'] = data_df['Runtime'].str.replace(r'\D', '') # removing str 'min' from each value
data_df['Runtime'] = data_df['Runtime'].astype(int) # converting the str values to ints

/tmp/ipykernel_44/2457863464.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_df['Runtime'] = data_df['Runtime'].str.replace(r'\D', '') # removing str 'min' from each value


In [169]:
shortest_rt = data_df['Runtime'].min()
print("The shortest runtime is: "+str(shortest_rt)+" minutes")
longest_rt = data_df['Runtime'].max()
print("The longest runtime is: "+str(longest_rt)+" minutes")

The shortest runtime is: 45 minutes
The longest runtime is: 321 minutes


In [170]:
shortest_rt_IMDB_rating = data_df['IMDB_Rating'][data_df['Runtime'] == shortest_rt]
print("Shortest runtime IMDB rating: "+str(float(shortest_rt_IMDB_rating.values)))
longest_rt_IMDB_rating = data_df['IMDB_Rating'][data_df['Runtime'] == longest_rt]
print("Longest runtime IMDB rating: "+str(float(longest_rt_IMDB_rating.values)))

Shortest runtime IMDB rating: 8.2
Longest runtime IMDB rating: 8.2


In [171]:
# x is the movie with the shorter runtime, y is the movie with the longer runtime
def compare_values(x, y):
    if x > y:
        return "The movie with the shortest runtime has the higher IMDB rating"
    elif y > x:
        return "The movie with the longest runtime has the higher IMDB rating"
    else:
        return "Both movies have the exact same IMDB rating"

result = compare_values(float(longest_rt_IMDB_rating.values), float(shortest_rt_IMDB_rating.values))
print(result)

shortest_rt_meta_score = data_df['Meta_score'][data_df['Runtime'] == shortest_rt]
print("Shortest runtime movie meta score: "+str(shortest_rt_meta_score.values))
print("Shortest runtime movie has no meta score\n")


data_df2 = data_df.drop(labels=194, axis=0)
shortest_rt_2 = data_df2['Runtime'].min()
print("The next shortest runtime is: "+str(shortest_rt_2))

Both movies have the exact same IMDB rating
Shortest runtime movie meta score: ['']
Shortest runtime movie has no meta score

The next shortest runtime is: 64


In [172]:
shortest_rt_meta_score_2 = data_df['Meta_score'][data_df['Runtime'] == shortest_rt_2]
print("The next shortest runtime movie meta score: "+str(float(shortest_rt_meta_score_2.values)))

longest_rt_meta_score = data_df['Meta_score'][data_df['Runtime'] == longest_rt]
print("Longest runtime movie meta score: "+str(float(longest_rt_meta_score.values)))

The next shortest runtime movie meta score: 80.0
Longest runtime movie meta score: 89.0


In [173]:
# x is the movie with the shorter runtime, y is the movie with the longer runtime
def compare_values_2(x, y):
    if x > y:
        return "The movie with the shortest runtime has the higher meta score"
    elif y > x:
        return "The movie with the longest runtime has the higher meta score"
    else:
        return "Both movies have the exact same meta score"
    
result_2 = compare_values_2(float(shortest_rt_meta_score_2.values), float(longest_rt_meta_score.values))
print(result_2)

The movie with the longest runtime has the higher meta score


In [174]:
# Follow up questions:
# What is the runtime of the movie with the highest meta score?
# How does it compare to the highest/lowest runtime movies?
highest_ms = data_df['Meta_score'].max()
highest_ms_mov = data_df[data_df['Meta_score'] == highest_ms]
avg_runtime = highest_ms_mov['Runtime'].mean()
print("The average runtime of the movies with the joint highest meta score is: "+str(avg_runtime))

The average runtime of the movies with the joint highest meta score is: 94.25


# **8.**
**Which director has the highest IMDB score? Which director has the highest Meta score?**

In [175]:
#Top 10 Directors by No of Votes on IMDB
print("Top 10 Directors by average number of votes on IMDb:")
director_stats = director_stats.sort_values(by=['avg No_of_Votes'], ascending=False)
director_stats.head(10)

Top 10 Directors by average number of votes on IMDb:


,Movie_Count,avg IMDB_Rating,avg No_of_Votes
Director,,,
Frank Darabont,2,8.95,"1,745,452.00"
Lana Wachowski,1,8.70,"1,676,426.00"
Christopher Nolan,8,8.46,"1,447,293.12"
George Lucas,1,8.60,"1,231,473.00"
Peter Jackson,5,8.40,"1,229,715.80"
Irvin Kershner,1,8.70,"1,159,315.00"
James McTeigue,1,8.20,"1,032,749.00"
Quentin Tarantino,8,8.18,"1,015,401.00"
Gore Verbinski,1,8.00,"1,015,122.00"


In [176]:
#Find movies directed by Darabont
Darabont = data_df.loc[data_df['Director'].str.contains("Frank Darabont")]
Darabont

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Movie_Count
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142,Drama,9.30,Two imprisoned men bond over a number of years...,80,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,"2,343,110.00","28,341,469",1
25,https://m.media-amazon.com/images/M/MV5BMTUxMz...,The Green Mile,1999,A,189,"Crime, Drama, Fantasy",8.60,The lives of guards on Death Row are affected ...,61,Frank Darabont,Tom Hanks,Michael Clarke Duncan,David Morse,Bonnie Hunt,"1,147,794.00","136,801,374",1


In [177]:
data_df.replace("", np.nan, inplace=True)
data_df = data_df.dropna()
data_df['Meta_score'] = data_df['Meta_score'].astype(int)
director_stats['Meta_score'] = data_df[['Director','Meta_score']].groupby(['Director']).mean()
director_stats = director_stats.rename(columns={"Meta_score" : "avg Meta_score"})

In [178]:
#Top 10 Directors by average Meta Score
print("Top 10 Directors by average Meta Score:\n")
director_stats = director_stats.sort_values(by=['avg Meta_score'], ascending=False)
director_stats.head(10)

Top 10 Directors by average Meta Score:



,Movie_Count,avg IMDB_Rating,avg No_of_Votes,avg Meta_score
Director,,,,
Ingmar Bergman,7,8.14,"72,296.14",100.00
Orson Welles,2,8.15,"250,891.00",99.50
Michael Curtiz,2,8.20,"284,634.00",98.50
Guillermo del Toro,1,8.20,"618,623.00",98.00
Joseph L. Mankiewicz,2,8.10,"82,643.50",98.00
Charles Chaplin,6,8.33,"138,907.00",97.50
Akira Kurosawa,10,8.22,"94,159.90",97.00
Sam Peckinpah,1,7.90,"77,401.00",97.00
Carol Reed,1,8.10,"158,731.00",97.00
